# Setting up Docker on Windows machine

https://docs.docker.com/docker-for-windows/install-windows-home/
1. Windows 10 64-bit: Pro, Enterprise, or Education (Build 16299 or later).
2. Hyper-V and Containers Windows features must be enabled.

After installation:
3. Enable WSL 2 on host computer: https://docs.microsoft.com/en-us/windows/wsl/install-win10
4. Enable WSL 2 based engine in Docker settings

Note: Docker containers do not have unique kernels. They are linked to what the host computer contains so if using Windows, Docker must use WSL 2. The kernel used in these instructions is Ubuntu 20.04 (Linux 4.19.104-microsoft-standard x86_64).

# Setting up Calypso Tracking on local machine - Docker: CentOS 7

## 1. Setting up Docker container and local CVMFS


---
Making a Dockerfile. This Docker file will give the Docker container "privileged" access meaning that it can access processes and devices on the host computer. By default containers are "unprivileged." This higher access is required to setup CVMFS on step 1.17.

1. `touch Dockerfile` (no file extension)
2. Paste content:
```
FROM centos:7
ENV container docker
RUN yum -y update; yum clean all
RUN yum -y install systemd; yum clean all; \
(cd /lib/systemd/system/sysinit.target.wants/; for i in *; do [ $i == systemd-tmpfiles-setup.service ] || rm -f $i; done); \
rm -f /lib/systemd/system/multi-user.target.wants/*;\
rm -f /etc/systemd/system/*.wants/*;\
rm -f /lib/systemd/system/local-fs.target.wants/*; \
rm -f /lib/systemd/system/sockets.target.wants/*udev*; \
rm -f /lib/systemd/system/sockets.target.wants/*initctl*; \
rm -f /lib/systemd/system/basic.target.wants/*;\
rm -f /lib/systemd/system/anaconda.target.wants/*;
VOLUME [ "/sys/fs/cgroup" ]
CMD ["/usr/sbin/init"]
``` 
3. Navigate to Dockerfile directory

NOTE: Step 4 can only be run in the default CMD app and not Powershell or Visual Studio.

4. `docker build --rm -t centos7-systemd - < Dockerfile`
5. `docker run --privileged -it -e container=docker -v /sys/fs/cgroup:/sys/fs/cgroup --name <container_name> centos7-systemd /usr/sbin/init`
  - NOTE: Make sure to set the container_name of your choice
  - Following step 6, you should see multiple "OK" symbols. 
<center>
<img src='https://drive.google.com/uc?export=view&id=1ALFBMWM1p6sK4CQN-kJBNqhS9aTL04sn' width="500px"/>
</center>
  - If this 'Failed' message appears, complete step 7 first and then follow the steps immediately below
    >1. `touch /etc/sysconfig/network`
    2. `systemctl restart network`
    3. Confirm network is active with: `systemctl status network.service`
<center>
<img src='https://drive.google.com/uc?export=view&id=1-p2B-kdJyqhSpxMwTvKyCAi7ZssGfAD_' width="500px"/>
</center>

7. Open a new shell window(you can close the first shell)
10. `Docker exec -it <container_name> /bin/bash`
11. `yum update`
12. `yum install -y centos-release-scl devtoolset-8 sudo vim`
15. `sudo yum install -y https://ecsft.cern.ch/dist/cvmfs/cvmfs-release/cvmfs-release-latest.noarch.rpm`
16. `sudo yum install -y cvmfs`
1. `cd /etc/cvmfs`
1. `vi default.local`
1. Paste content: 
```
CVMFS_REPOSITORIES=atlas.cern.ch,atlas-condb.cern.ch,atlas-nightlies.cern.ch,sft.cern.ch,faser.cern.ch
CVMFS_QUOTA_LIMIT=61440
CVMFS_HTTP_PROXY=DIRECT
```
1. `:wq`
  - NOTE: At this point, the container will take up ~232MB (virtual 484MB) of space
17. `cvmfs_config setup`
18. Check cvmfs installation with: `cvmfs_config chksetup`
19. Should output `OK`

Autofs version: autofs-5.0.7-109.el7.x86_64

If autofs reports an error, check that the container is privileged and WSL 2 is enabled for Windows.
<!--https://scriptthe.net/2015/02/05/autofs-in-docker-containers/-->




## 2. Setting up Athena


---
Running Environment: Windows 10 host with CentOS 7 in Docker container

Assuming CVMFS has already been setup in the Docker container

1. ```export ATLAS_LOCAL_ROOT_BASE=/cvmfs/atlas.cern.ch/repo/ATLASLocalRootBase```
2. ```sudo yum install -y man```
<!--this step takes a while-->
3. ```source ${ATLAS_LOCAL_ROOT_BASE}/user/atlasLocalSetup.sh```
  - After this step, /cvmfs/ directory should be non-empty

<center>
<img src='https://drive.google.com/uc?export=view&id=1-RanFXZ1k2r15RJG9DwoFK--f7lR304r' width="900px"/>
</center>

4. `asetup --input=calypso/asetup.faser master,Athena,r09-14`
  - Run this step from directory that contains calypso
  - Depending on your network connection, this step may take a while
<!--fsetup --input=calypso/asetup.faser Athena,22.0.18-->

## 3. Setting up Calypso


---
Assuming ATLAS environment already set in Docker container

1. ```sudo yum install -y git make cmake glibc-devel mesa-libGL-devel qt5-qtbase```
2. Go to https://gitlab.cern.ch/faser/calypso and make a personal fork
3. ```git clone --recursive [repo_link_of_personal_fork]```
4. ```mkdir build; cd build```
<!--gives access to missing libEGL.so and libGL.so-->
7. ```sudo ln /usr/lib64/libEGL.so.1 /lib64/libEGL.so```
8. ```sudo ln /usr/lib64/libGL.so.1 /lib64/libGL.so```
<!--https://centos.pkgs.org/7/centos-x86_64/libuuid-2.23.2-63.el7.x86_64.rpm.html-->
1. ```sudo ln /usr/lib64/libuuid.so.1 /usr/lib64/libuuid.so```
7. ```cmake -DCMAKE_INSTALL_PREFIX=../run ../calypso ; make ; make install```

cmake3:
- yum install epel-release
- yum install cmake3
- sudo yum remove cmake
- sudo ln -s /usr/bin/cmake3 /usr/bin/cmake

### !!!Currently stuck here as of Oct-3!!!
```
[ 41%] Linking CXX shared library ../../generic/lib/libFaserEventStorageLib.so
Detaching debug info of libGeoModelSvc.so into libGeoModelSvc.so.dbg
[ 41%] Built target GeoModelSvc
CMakeFiles/FaserEventStorageLib.dir/src/DataReaderController.cxx.o: In function `FaserEventStorage::DataReaderController::dataMB_InFileSequence()':
/cvmfs/atlas.cern.ch/repo/sw/tdaq/tdaq-common/tdaq-common-04-02-00/installed/include/ers/ers.h:67: undefined reference to `ers::Configuration::instance()'
CMakeFiles/FaserEventStorageLib.dir/src/DataReaderController.cxx.o: In function `FaserEventStorage::DataReaderController::dataMB_InFileSequence()':
/calypso/Event/FaserEventStorage/src/DataReaderController.cxx:607: undefined reference to `ers::LocalContext::LocalContext(char const*, char const*, int, char const*, bool)'
```

```
/cvmfs/atlas-nightlies.cern.ch/repo/sw/master_Athena_x86_64-centos7-gcc8-opt/sw/lcg/releases/qt5/5.12.4-3eab3/x86_64-centos7-gcc8-opt/include/QtGui/qopengl.h:141:13: fatal error: GL/gl.h: No such file or directory
 #   include <GL/gl.h>
```


